In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %matplotlib widget

In [ ]:
from __future__ import annotations

import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data
)

from cbc import CBCProjection, cp_triangle_norm_sq
from robust_voltage_control import (
    VoltPlot,
    np_triangle_norm,
    robust_voltage_control
)

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# hide top and right splines on plots
# plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

## Calculate minimum q-limits needed to keep voltage within bounds

Answer: ±0.23757973725624573

In [ ]:
net = create_56bus()
R, X = create_RX_from_net(net, noise=0)
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape

In [ ]:
v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]
Vpar_min = np.min(vpars, axis=1)  # shape [n]
Vpar_max = np.max(vpars, axis=1)  # shape [n]

In [ ]:
batch_size = 69  # choose a number that T is divisible by
assert T % batch_size == 0

vpars_param = cp.Parameter([batch_size, n])
qcs = cp.Variable([batch_size, n])
obj = cp.Minimize(cp.max(cp.abs(qcs)))
constrs = [
    v_min <= qcs @ X + vpars_param,
    qcs @ X + vpars_param <= v_max]
prob = cp.Problem(obj, constrs)

In [ ]:
bounds = []
for i in tqdm(range(T // batch_size)):
    vpars_param.value = vpars[i*batch_size:(i+1)*batch_size]
    prob.solve(solver=cp.MOSEK, warm_start=True)
    # tqdm.write(f'{i}, {prob.value}')
    bounds.append(prob.value)
print(np.max(bounds))

## Plot $\left\|\hat{X}-X\right\|_\triangle$ as a function of line-parameter noise

In [ ]:
net = create_56bus()
_, X_true = create_RX_from_net(net, noise=0)
norm_true = np_triangle_norm(X_true)
print(f'||X||△ = {norm_true}')

nsamples = 50
noises = np.arange(0.01, 1, 0.01)
norms = np.zeros([len(noises), nsamples])
for i, noise in tqdm(enumerate(noises)):
    for j in range(nsamples):
        _, Xhat = create_RX_from_net(net, noise=noise, check_pd=True, seed=None)
        norms[i,j] = np_triangle_norm(X_true - Xhat)

In [ ]:
norms_mean = norms.mean(axis=1)
norms_std = norms.std(axis=1)

fig, ax = plt.subplots(1, 1, figsize=(5,4), tight_layout=True)
ax.plot(noises, norms_mean)
ax.fill_between(noises, norms_mean - norms_std, norms_mean + norms_std, alpha=0.5)
for j in range(nsamples):
    ax.scatter(noises, norms[:,j], s=1, color='b', alpha=0.1)
ax.set(xlabel='Uniform noise', ylabel='$||\hat{X}-X||$')

ax2 = ax.twinx()
y_min, y_max = ax.get_ylim()
ax2.set_ylim(y_min / norm_true, y_max / norm_true)
ax2.set(ylabel='$||\hat{X}-X|| / ||X||$')